In [1]:
import sys
import os

# Add the parent directory of 'vegetation-period-NDVI' to the sys.path
sys.path.append(os.path.abspath(os.path.join('..')))


In [2]:
import ee
import geemap
from time_series import extract_time_ranges, get_harmonic_ts
from vegetation_period_extraction import (
    get_crop_veg_period,
    create_binary_ndvi_indicator,
    create_binary_mask,
)
from typing import List

In [3]:
ee.Initialize(project="thurgau-irrigation")

## Exporting the vegetation period images as an asset to the project

### Define the Area of Interest

In [4]:
cantonal_borders_asset = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

aoi_name = "Thurgau"

### Define the years of interest

In [5]:
years = [2019, 2023]

### Export the Vegetation Periods images to the project assets

In [6]:
for year in years:

    print(f"Processing year {year}")

    time_intervals = extract_time_ranges([f"{year}-03-01", f"{year}-10-31"], 15)

    image_to_export = get_crop_veg_period(year, aoi_buffered, time_intervals)

    task_name = f"Thurgau_crop_veg_period_{year}"
    asset_id = f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{year}"

    bands_to_export = ["firstStart", "firstEnd", "secondStart", "secondEnd", "isDoubleCropping"]

    image_to_export = image_to_export.select(bands_to_export)


    task = ee.batch.Export.image.toAsset(
        image=image_to_export,
        description=task_name,
        assetId=asset_id,
        region=aoi_buffered,
        scale=10,
        maxPixels=1e13,
    )

    task.start()

    print(f"Exporting {task_name} to {asset_id}")

Processing year 2019
Exporting Thurgau_crop_veg_period_2019 to projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2019
Processing year 2023
Exporting Thurgau_crop_veg_period_2023 to projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2023


## General Code Testing. Ignore this part

In [13]:
year = 2019

# Define the time range for analysis
start_date = f"{year}-03-01"
end_date = f"{year}-10-31"

# Create time intervals
time_intervals = extract_time_ranges([start_date, end_date], 15)

# Get the vegetation period estimation
veg_period_image = get_crop_veg_period(year, aoi_buffered, time_intervals)

In [14]:
# type(veg_period_image)

In [15]:
double_band = veg_period_image.select('isDoubleCropping')

# Create a map
Map = geemap.Map(zoom=10)
Map.centerObject(aoi_buffered)
# Add the 'double' band to the map
Map.addLayer(double_band, {'min': 0, 'max': 1, 'palette': ['blue', 'green', 'red']}, 'Double Band')

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [16]:
filtered_sentinel_data = get_harmonic_ts(year=year, aoi=aoi_buffered, time_intervals=time_intervals)

In [17]:
filtered_sentinel_data

{'fitted_data': <ee.imagecollection.ImageCollection at 0x2891a8f50>,
 'regression_coefficients': <ee.image.Image at 0x2896ec690>,
 'phase_amplitude': <ee.image.Image at 0x28912e350>}

In [18]:
Map = geemap.Map()


# Add the layer to the map.
ndvi_image = ee.Image(filtered_sentinel_data.get("fitted_data").toList(99).get(7))
Map.centerObject(aoi_buffered, 13)
ndvi_params = {'bands': ['rmse'], 'min': 0, 'max': 1, 'palette': ['white', 'green']}
Map.addLayer(ndvi_image, ndvi_params, 'NDVI FIRST', True)

# Display the map.
Map

Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…